LIbraries (MUST RUN THIS CELL)

In [0]:
# 1. Install UI and Embedding models
%pip install gradio==3.50.2 sentence-transformers --quiet

# 2. Install Utilities
%pip install spark-nlp nltk duckduckgo-search python-dotenv

# 3. Install LangChain Framework & Graph (Letting 'core' resolve automatically)
%pip install langchain==0.3.14 langgraph==0.2.60

# 4. Install LangChain Integrations (Letting 'pinecone-client' resolve automatically)
%pip install langchain-google-genai==2.0.7 langchain-huggingface==0.1.2 langchain-pinecone==0.2.0 langchain-groq==0.2.2

# 5. Apply changes
dbutils.library.restartPython()

# Hotel Intelligence System - Gradio Interface

This interface provides an interactive way to query the Hotel Intelligence System using multiple AI agents.

## Features

- **Competitor Analysis**: Deep NLP analysis comparing your property's reviews vs similar properties across 20 topics
- **Feature Impact Analysis**: Linear regression model identifying which features most impact ratings
- **Market Intelligence**: Web scraping and sentiment analysis for competitive insights
- **Review Analysis**: Topic-specific review analysis with evidence quotes

## Performance Notes

- In order to check if a reponse has arrived - please click on "Check Status" button.
- NLP and LR analyses take approximately **7 minutes each**
- Charts are automatically generated and linked in responses

In [0]:
"""
Gradio Chat Interface for Databricks
"""

import threading
import time
import copy
import sys
import os
import builtins
import gradio as gr
from PIL import Image as PILImage
from io import BytesIO
import base64

# Get the current user's email dynamically
current_user = spark.sql("SELECT current_user()").collect()[0][0]
# Construct the base path dynamically
base_path = f"/Workspace/Users/{current_user}/hotel-intelligence-system"
sys.path.insert(0, f"{base_path}")
sys.path.insert(0, f"{base_path}/agents")

builtins.dbutils = dbutils  # Required for Databricks

# ==============================================
# CONFIGURATION
# ==============================================

HOTEL_ID = "ABB_40458495"
HOTEL_NAME = "Rental unit in Broadbeach"  
CITY = "Broadbeach"

from agents.coordinator import LangGraphCoordinator
coordinator = LangGraphCoordinator(HOTEL_ID, HOTEL_NAME, CITY)
state = coordinator.get_initial_state()

# Thread-safe global storage
captured_charts = []
state_lock = threading.Lock()
job_lock = threading.Lock()

current_job = {
    "running": False,
    "progress": "",
    "result": None,
    "error": None,
    "start_time": None,
    "thread_id": None
}

# ==============================================
# CHART CAPTURE PATCH 
# ==============================================

_patched_modules = []

def _capture_charts(result):
    global captured_charts
    
    charts = result.get("charts", {})
    if not charts:
        ui_artifacts = result.get("ui_artifacts", {})
        charts = ui_artifacts.get("charts", {}) if isinstance(ui_artifacts, dict) else {}

    captured_charts = []
    for name, b64 in charts.items():
        if isinstance(b64, dict):
            b64 = b64.get("data") or b64.get("base64") or b64.get("image")
        
        if b64 and isinstance(b64, str):
            try:
                img = PILImage.open(BytesIO(base64.b64decode(b64)))
                captured_charts.append(img)
            except Exception as e:
                pass  # Silently fail on bad image data to prevent crash

def _patch_lr(module, module_name):
    original = module.run_lr_analysis

    def patched_lr(hotel_id, timeout_seconds=1200):
        result = original(hotel_id, timeout_seconds)
        _capture_charts(result)
        return result

    module.run_lr_analysis = patched_lr
    _patched_modules.append(module_name)

try:
    import databricks_tools
    _patch_lr(databricks_tools, "databricks_tools")
except Exception:
    pass

try:
    from agents import databricks_tools as agents_databricks_tools
    _patch_lr(agents_databricks_tools, "agents.databricks_tools")
except Exception:
    pass


# ==============================================
# ASYNC JOB HANDLING 
# ==============================================

def run_analysis_thread(query: str, thread_state: dict):
    global captured_charts, current_job
    
    with job_lock:
        current_job["running"] = True
        current_job["progress"] = "Initializing..."
        current_job["result"] = None
        current_job["error"] = None
        current_job["start_time"] = time.time()
        current_job["thread_id"] = threading.current_thread().ident
        captured_charts = []
    
    result_state = None
    
    try:
        with job_lock:
            current_job["progress"] = "Extracting entities from query..."
        time.sleep(0.5)
        
        with job_lock:
            current_job["progress"] = "Routing to specialist agent(s)..."
        time.sleep(0.5)
        
        with job_lock:
            current_job["progress"] = "Agent working (may take 15-20 min for deep analysis)..."
        
        start_exec = time.time()
        response, result_state = coordinator.run(query, thread_state)
        exec_time = time.time() - start_exec
        
        with job_lock:
            current_job["result"] = response
            current_job["progress"] = f"Complete! (took {exec_time/60:.1f} min)"
        
        with state_lock:
            global state
            state = result_state
            
    except Exception as e:
        import traceback
        
        with job_lock:
            current_job["error"] = str(e)
            current_job["progress"] = f"Error: {str(e)[:100]}"
    
    finally:
        with job_lock:
            current_job["running"] = False


def start_analysis(message: str, history: list):
    global current_job, state
    
    if not message.strip():
        return "", history, [], "⚠️ Please enter a question"
    
    with job_lock:
        if current_job["running"]:
            elapsed = int(time.time() - current_job["start_time"]) if current_job["start_time"] else 0
            return "", history, [], f"⏳ Analysis already in progress... ({elapsed}s)\n{current_job['progress']}"
    
    history = history + [(message, "⏳ Processing... This may take 15-20 minutes for deep analysis.\n\nClick 'Check Status' to see progress.")]
    
    with state_lock:
        thread_state = copy.deepcopy(state)
    
    thread = threading.Thread(
        target=run_analysis_thread, 
        args=(message, thread_state),
        name=f"Analysis-{int(time.time())}"
    )
    thread.daemon = True
    thread.start()
    
    time.sleep(0.5)
    
    return "", history, [], "⏳ Analysis started in background..."


def check_status(history: list):
    global current_job, captured_charts
    
    if not history:
        return history, [], "No analysis running"
    
    with job_lock:
        is_running = current_job["running"]
        progress = current_job["progress"]
        result = current_job["result"]
        error = current_job["error"]
        start_time = current_job["start_time"]
        thread_id = current_job["thread_id"]
    
    elapsed = int(time.time() - start_time) if start_time else 0
    mins, secs = divmod(elapsed, 60)
    
    if is_running:
        progress_bar = "█" * min(20, elapsed // 30) + "░" * max(0, 20 - elapsed // 30)
        status = f"⏳ Running ({mins}m {secs}s elapsed)\n[{progress_bar}]\n\nThread ID: {thread_id}\nStatus: {progress}"
        return history, captured_charts, status
    
    elif result:
        if history and history[-1][1] and history[-1][1].startswith("⏳"):
            history[-1] = (history[-1][0], result)
        status = f" Complete! ({mins}m {secs}s total)\n📊 Charts: {len(captured_charts)}\nClick 'Check Status' again to refresh charts."
        return history, captured_charts, status
    
    elif error:
        if history and history[-1][1] and history[-1][1].startswith("⏳"):
            history[-1] = (history[-1][0], f" Error: {error}")
        status = f" Error after {mins}m {secs}s"
        return history, captured_charts, status
    
    else:
        return history, captured_charts, "Ready - enter a question"


def clear_chat():
    global state, current_job, captured_charts
    
    with state_lock:
        state = coordinator.get_initial_state()
    
    with job_lock:
        current_job = {
            "running": False,
            "progress": "",
            "result": None,
            "error": None,
            "start_time": None,
            "thread_id": None
        }
    
    captured_charts = []
    
    return [], [], "Cleared - ready for new questions"


# ==============================================
# GRADIO INTERFACE
# ==============================================

with gr.Blocks(title=f"Hotel Intelligence: {HOTEL_NAME}") as demo:
    gr.Markdown(f"# Hotel Intelligence: {HOTEL_NAME}")
    gr.Markdown(f"""    
    **Hotel:** {HOTEL_NAME} (ID: {HOTEL_ID})  
    **City:** {CITY}
    """)
    
    chatbot = gr.Chatbot(height=400, label="Conversation")
    
    with gr.Row():
        msg = gr.Textbox(
            placeholder="Ask: 'What features should I improve to increase my rating?'", 
            show_label=False, 
            scale=4
        )
        send_btn = gr.Button("Send", variant="primary")
    
    with gr.Row():
        check_btn = gr.Button("Check Status", variant="secondary")
        clear_btn = gr.Button("Clear Chat", variant="stop")
    
    status_box = gr.Textbox(
        label="Status", 
        value="Ready - enter a question",
        interactive=False,
        lines=5
    )
    
    chart_gallery = gr.Gallery(
        label="📊 Analysis Charts", 
        columns=2, 
        height=300, 
        visible=True
    )
    
    send_btn.click(fn=start_analysis, inputs=[msg, chatbot], outputs=[msg, chatbot, chart_gallery, status_box])
    msg.submit(fn=start_analysis, inputs=[msg, chatbot], outputs=[msg, chatbot, chart_gallery, status_box])
    check_btn.click(fn=check_status, inputs=[chatbot], outputs=[chatbot, chart_gallery, status_box])
    clear_btn.click(fn=clear_chat, inputs=[], outputs=[chatbot, chart_gallery, status_box])

demo.queue()
demo.launch(share=True)